In [ ]:
# 我的专属全能云端下载器
## 功能: 支持HTTP直链, BT磁力链接, YouTube视频下载

**使用流程:**
1.  **首次运行**: 从上到下依次运行所有单元格。如果【步骤1】完成后，后续步骤报错`ModuleNotFoundError`，请手动点击菜单栏的 **`代码执行程序`** -> **`重启会话`**，然后再从头依次运行所有单元格。
2.  **日常使用**: 从上到下依次运行所有单元格，然后直接在【步骤4】的图形界面中操作即可。

In [ ]:
# ===================================================================
# 步骤 1: 安装所有必需的工具 (修正版)
# ===================================================================
print("🚀 开始安装所有必需的下载工具...")

# 使用 apt-get 在系统层面安装 aria2c 命令行工具
print("正在安装 aria2c 命令行工具...")
!apt-get -qq update
!apt-get -qq install -y aria2

# 使用 pip 安装 libtorrent (确保版本兼容) 和其他Python库
print("正在安装 libtorrent, aria2p, yt-dlp 等Python库...")
!pip -q install libtorrent aria2p yt-dlp

# 检查关键工具是否安装成功
print("\n--- 安装后检查 ---")
!which aria2c
!python -c "import libtorrent; print('libtorrent-ok')"
!which yt-dlp
print("--------------------")

print("\n✅ 所有工具安装完毕！")

In [ ]:

# ===================================================================
# 步骤 2: 导入Python库并定义核心下载函数
# ===================================================================
print("📚 正在导入库并准备核心功能...")

import os
import time
import libtorrent as lt
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output
import aria2p

# --- 定义HTTP下载函数 (使用aria2p库) ---
def start_http_download(links, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"⬇️ 开始HTTP下载，保存到: {save_path}")
    link_list = [link.strip() for link in links.strip().split('\n') if link.strip()]
    if not link_list:
        print("⚠️ 链接列表为空，请提供有效的下载链接。")
        return

    # 初始化aria2客户端
    aria2 = aria2p.API(
        aria2p.Client(
            host="http://localhost",
            port=6800,
            secret=""
        )
    )
    # 启动aria2守护进程
    get_ipython().system(f'aria2c --enable-rpc --rpc-listen-all=true --rpc-allow-origin-all -c -x 16 -s 16 -k 1M -d "{save_path}" --daemon=true --quiet=true')
    time.sleep(2) # 等待守护进程启动

    for link in link_list:
        try:
            aria2.add_uris([link])
        except Exception as e:
            print(f"添加任务失败: {e}")
            continue
    print(f"\n🎉 {len(link_list)} 个HTTP下载任务已提交给aria2处理。")
    print("ℹ️ HTTP下载在后台进行，您可继续添加其他任务。")


# --- 定义BT下载函数 (不变) ---
def start_bt_download(magnet_link, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"🧲 开始BT下载，保存到: {save_path}")
    if not magnet_link.startswith("magnet:?"):
        print("⚠️ 无效的磁力链接。")
        return

    ses = lt.session({'listen_interfaces': '0.0.0.0:6881'})
    params = {'save_path': save_path}
    handle = lt.add_magnet_uri(ses, magnet_link, params)

    print("正在获取种子元数据...")
    while not handle.has_metadata():
        time.sleep(1)
    print(f"元数据获取成功！种子名称: {handle.status().name}")

    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='下载中:')
    display(progress_bar)

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        progress_bar.value = s.progress * 100
        progress_bar.description = f"下载中: {s.progress*100:.2f}% ({s.download_rate/1000:.2f} kB/s) | 状态: {s.state}"
        time.sleep(1)

    progress_bar.description = f"✅ 下载完成: {handle.status().name}"
    print(f"\n🎉 BT任务 '{handle.status().name}' 下载完成！")


# --- 定义YouTube下载函数 (不变) ---
def start_youtube_download(url, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"📹 开始YouTube/视频下载，保存到: {save_path}")
    if not url.strip():
        print("⚠️ 请提供有效的视频链接。")
        return

    command = f'yt-dlp -P "{save_path}" -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best" --merge-output-format mp4 "{url}"'
    get_ipython().system(command)
    print("\n🎉 视频下载任务已提交给yt-dlp处理。")

print("✅ 核心功能已准备就绪！")

In [ ]:
# ===================================================================
# 步骤 3: 连接到您的 Google Drive
# ===================================================================
print("🔗 正在请求连接到您的 Google Drive...")
drive.mount('/content/drive')

In [ ]:
# ===================================================================
# 步骤 4: 创建并显示下载器图形界面 (GUI)
# ===================================================================

# --- 默认保存路径 ---
default_save_path = '/content/drive/MyDrive/Downloads'

# --- 创建HTTP下载界面组件 ---
http_links_input = widgets.Textarea(
    placeholder='在此处输入一个或多个HTTP/HTTPS直链，每行一个。',
    layout={'height': '100px', 'width': '98%'}
)
http_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
http_button = widgets.Button(description="开始HTTP下载", button_style='success', icon='download')
http_output = widgets.Output()

def on_http_button_clicked(b):
    with http_output:
        clear_output(wait=True)
        start_http_download(http_links_input.value, http_save_path_input.value)
http_button.on_click(on_http_button_clicked)
http_tab = widgets.VBox([http_save_path_input, http_links_input, http_button, http_output])


# --- 创建BT下载界面组件 ---
bt_magnet_input = widgets.Text(placeholder='在此处输入一个磁力链接。', layout={'width': '98%'})
bt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
bt_button = widgets.Button(description="开始BT下载", button_style='info', icon='magnet')
bt_output = widgets.Output()

def on_bt_button_clicked(b):
    with bt_output:
        clear_output(wait=True)
        start_bt_download(bt_magnet_input.value, bt_save_path_input.value)
bt_button.on_click(on_bt_button_clicked)
bt_tab = widgets.VBox([bt_save_path_input, bt_magnet_input, bt_button, bt_output])


# --- 创建YouTube下载界面组件 ---
yt_url_input = widgets.Text(placeholder='在此处输入一个YouTube或其他支持的视频链接。', layout={'width': '98%'})
yt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
yt_button = widgets.Button(description="开始视频下载", button_style='danger', icon='youtube')
yt_output = widgets.Output()

def on_yt_button_clicked(b):
    with yt_output:
        clear_output(wait=True)
        start_youtube_download(yt_url_input.value, yt_save_path_input.value)
yt_button.on_click(on_yt_button_clicked)
yt_tab = widgets.VBox([yt_save_path_input, yt_url_input, yt_button, yt_output])


# --- 创建并显示选项卡界面 ---
tab = widgets.Tab()
tab.children = [http_tab, bt_tab, yt_tab]
tab.set_title(0, 'HTTP / HTTPS')
tab.set_title(1, 'BitTorrent (磁力链接)')
tab.set_title(2, 'YouTube & More')

print("✅ 下载器界面已准备好，请在下方操作。")
display(tab)